In [12]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan  5 20:44:42 2023

@author: 18172
"""

# -*- coding: utf-8 -*-
"""
Created on Thu Jan  5 16:59:18 2023

@author: 18172
"""

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ast

week1 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week1.csv')
week2 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week2.csv')
week3 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week3.csv')
week4 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week4.csv')
week5 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week5.csv')
week6 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week6.csv')
week7 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week7.csv')
week8 = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\week8.csv')
pff = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\pffScoutingData.csv')

data = week1.append(week2)
data = data.append(week3)
data = data.append(week4)
data = data.append(week5)
data = data.append(week6)
data = data.append(week7)
data = data.append(week8)

data = data.merge(pff, how = 'left')

data['uId'] = data['gameId'].to_string() + "-" + data['playId'].to_string()

idxs = (data
        .loc[data['event'] == 'ball_snap',
                 'frameId']
        .index
        .values)

x = [(idxs+x).tolist() for x in range(0,24)]
idxs = [item for sublist in x for item in sublist]

df = data.loc[idxs]

_los = (data
        .loc[(data['team']=='football') &
             (data['frameId']==1),
             ['gameId', 'playId', 'x']]
        .rename(columns={'x':'los'}))

_ball = (data
        .loc[(data['team']=='football') &
             (data['frameId']==1),
             ['gameId', 'playId', 'y']]
        .rename(columns={'y':'cop'}))

df = df.merge(_los)
df = df.merge(_ball)

p_r = df[df['pff_role'] == "Pass Rush"]

def zones (row):
    rel_pos = row['cop'] - row['y']
    
    #add a line that accounts for non lineman, and sets their zone to something different
    #also need to determine what direction the defense is facing and adjust accordingly
    #rn I have 14 zones at half a yard each which is arbitrary
    pos = row["pff_positionLinedUp"] 
    corners = ["LCB", "RCB", "SCBiL", "SCBiR", "SCBL",  "SCBoL", "SCBoR", "SCBR"]
#     if pos in corners:
#         if row['playDirection'] == 'right':
#             if rel_pos<-3:
#                 return -100
#             else:
#                 return 100
#         if row['playDirection'] == 'left':
#             if rel_pos<-3:
#                 return 100
#             else:
#                 return -100
    
#     vert_pos = row['los'] - row['y']
    
#     if abs(vert_pos > 1.5):
#         return -2
    
    if row['playDirection'] == 'right':
        if (rel_pos < -3):
            return 1
        elif (rel_pos < -2.5):
            return 2
        elif (rel_pos < -2):
            return 3
        elif (rel_pos < -1.5):
            return 4
        elif (rel_pos < -1):
            return 5
        elif (rel_pos < -.5):
            return 6
        elif (rel_pos < 0):
            return 7
        elif (rel_pos < .5):
            return 8
        elif (rel_pos < 1):
            return 9
        elif (rel_pos < 1.5):
            return 10
        elif (rel_pos < 2):
            return 11
        elif (rel_pos < 2.5):
            return 12
        elif (rel_pos < 3):
            return 13
        elif (rel_pos >= 3):
            return 14
        return 'Other'
    else:
        if (rel_pos < -3):
            return 14
        elif (rel_pos < -2.5):
            return 13
        elif (rel_pos < -2):
            return 12
        elif (rel_pos < -1.5):
            return 1
        elif (rel_pos < -1):
            return 10
        elif (rel_pos < -.5):
            return 9
        elif (rel_pos < 0):
            return 8
        elif (rel_pos < .5):
            return 7
        elif (rel_pos < 1):
            return 6
        elif (rel_pos < 1.5):
            return 5
        elif (rel_pos < 2):
            return 4
        elif (rel_pos < 2.5):
            return 3
        elif (rel_pos < 3):
            return 2
        elif (rel_pos >= 3):
            return 1
        return 'Other'

p_r['zone'] = p_r.apply (lambda row: zones(row), axis=1)

def posdef (row):
    pos = row['pff_positionLinedUp']
    corners = ["LCB", "RCB", "SCBiL", "SCBiR", "SCBL",  "SCBoL", "SCBoR", "SCBR"]
    vert_pos = (row['los'] - row['x'])
    if row['playDirection'] == "right":
        vert_pos = vert_pos * -1
        
    if pos in corners:
        return(3)        
    elif vert_pos>1.5:
        return(2)
    else:
        return(1)
p_r['npos'] = p_r.apply (lambda row: posdef(row), axis=1)

current_timestamp = None
locations_at_timestamp = []
relative_locations_at_timestamp = {}

for index, row in p_r.iterrows():
    if row['time'] != current_timestamp:
        # New timestamp, so update the current timestamp and reset the list of locations
        current_timestamp = row['time']
        locations_at_timestamp = []
    pos = row["pff_positionLinedUp"] 
    corners = ["LCB", "RCB", "SCBiL", "SCBiR", "SCBL",  "SCBoL", "SCBoR", "SCBR"]
    vert_pos = (row['los'] - row['x'])
    if row['playDirection'] == "right":
        vert_pos = vert_pos * -1
    

    locations_at_timestamp.append(row['y'])
    
    # Step 3: Sort the list of locations at each timestamp
    locations_at_timestamp.sort()
    
    # Step 4: Assign a relative location to each player at each timestamp
    relative_locations = {}
    for i, location in enumerate(locations_at_timestamp):
        relative_locations[location] = i + 1
        
    # Store the relative locations for each player at this timestamp
    relative_locations_at_timestamp[current_timestamp] = relative_locations

counte = 0
count = 0
for index, row in p_r.iterrows():
    try:
        p_r.at[index, 'relative_location'] = relative_locations_at_timestamp[row['time']][row['y']]
        count += 1
    except KeyError:
        p_r.at[index, 'relative_location'] = 0
        counte += 1
        
        
        
def group_and_create_column(df, group_col, value_col):
  # Group the dataframe by the specified column
  grouped_df = df.groupby(group_col)
  
  # Create a new column with a list of all the values in the specified column for each group
  new_df = grouped_df[value_col].apply(list).reset_index()
  
  # Rename the column to include the name of the original column
  new_df = new_df.rename(columns={value_col: f"{value_col}_list"})
  
  return new_df

b = group_and_create_column(p_r, 'time', 'relative_location')
m = pd.merge(p_r, b, on = "time", how = "outer")
m.info()


del week1
del week2
del week3
del week4
del week5
del week6
del week7
del week8


C:\Users\18172\AppData\Local\Temp\ipykernel_13112\3293758015.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = week1.append(week2)
C:\Users\18172\AppData\Local\Temp\ipykernel_13112\3293758015.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(week3)
C:\Users\18172\AppData\Local\Temp\ipykernel_13112\3293758015.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(week4)
C:\Users\18172\AppData\Local\Temp\ipykernel_13112\3293758015.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(week5)
C:\Users\18172\AppData\Local\Temp\ipykernel_13112\3293758015.py:34: FutureWarning: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 870198 entries, 0 to 870197
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   gameId                  870198 non-null  int64  
 1   playId                  870198 non-null  int64  
 2   nflId                   870198 non-null  float64
 3   frameId                 870198 non-null  int64  
 4   time                    870198 non-null  object 
 5   jerseyNumber            870198 non-null  float64
 6   team                    870198 non-null  object 
 7   playDirection           870198 non-null  object 
 8   x                       870198 non-null  float64
 9   y                       870198 non-null  float64
 10  s                       870198 non-null  float64
 11  a                       870198 non-null  float64
 12  dis                     870198 non-null  float64
 13  o                       870198 non-null  float64
 14  dir                 

In [74]:
z = m[['gameId', 'playId', 'nflId', 'time',  'playDirection', 'x', 'relative_location', 'relative_location_list', 'npos']]

In [75]:
fb = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\full_front_back_data.csv')


In [76]:
def convert_list(x):
    if len(x) == 1:
        return [x]
    else:
        x = x[2:-2]  # remove the "c(" and ")" from the string
        x = x.replace('"', '')  # remove the quotation marks around the numbers
        return ast.literal_eval(f'[{x}]')  # wrap the string in square brackets to create a list
    
fb['all_front'] = fb['all_front'].apply(convert_list)
fb['all_back'] = fb['all_back'].apply(convert_list)
fb = fb.set_index('...1')
fb2 = fb[['all_front', 'all_back']]
z = z.join(fb2)
z['all_front'] = z['all_front'].fillna(0)
z['all_back'] = z['all_back'].fillna(0)


In [84]:
def identify_stunt(row):
    list1 = row["relative_location_list"]
    list2 = row['lag_relative_location_list']
    back = row['all_back']
    
    ##################################################################################################
    # * will indicate that farthest outside defender crosses over, or in other words is behind/in back
    ##################################################################################################
    if row['stunt'] == 0:
      return 0

    if back == 0:
        return 0
    len1 = len(list1)
    len2 = len(list2)
    
    if (len1 == 3):
        return 0
    if (len1 == 2):
        return 0
    
    
    stunt_ids = []
    
    # Loosely checking if lists are from same play
    if len1 != len2:
      return "Error: mismatching lists"

    if list1 != list2:
     
      # Turning (1,2,3,4,5) into (0,1,2,3,4) for ease of indexing
      list1 = [int(x - 1) for x in list1]
      list2 = [int(x - 1) for x in list2]
  
      # Reordering lists so the relative positions are in order
      index = [0] * len1
      for i in range(len1):
          index[list1[i]] = i
  
      # (1,2,3,4,5) -> (2,1,3,4,5), or (0,1,2,3,4) -> (1,0,2,3,4)
      # far left stunt: 1
      if (list2[index[0]] == 1) & (list2[index[1]] == 0):
          if ("1" in back) or (1 in back):
            return(1)
          else:
            return(2)
  
      # (1,2,3,4,5) -> (1,2,3,5,4), or (0,1,2,3,4) -> (0,2,1,4,3)
      # far right stunt: 2
      if (list2[index[-1]] == len1-2) & (list2[index[-2]] == len1-1):
          if (str(len1) in back) or (len1 in back):
            return(3)
          else:
            return(4)
  
      # (1,2,3,4,5) -> (3,2,1,4,5), or (0,1,2,3,4) -> (2,1,0,3,4)
      # far left-to-mid stunt: 3
      if (len1 > 2) & (list2[index[0]] in range(1,3)) & (list2[index[2]] == 0):
        return 7
#           if 1 in back:
#             return 7
#           else:
#             return 8
      
      # (1,2,3,4,5) -> (1,2,5,4,3), or (0,1,2,3,4) -> (0,1,4,3,2)
      # far right-to-mid stunt: 4
      if (len1 > 2) & (list2[index[-1]] in range(len1-3, len1-1)) & (list2[index[-3]] == len1-1):
            return 8
            
            
            #           if len1 in back:
#             return 9
#           else:
#             return 0
  
      # (1,2,3,4,5) -> (1,3,2,4,5), or (0,1,2,3,4) -> (0,2,1,3,4)
      # left-mid stunt: 5
      if (len1 > 4) & (list2[index[1]] == 2) & (list2[index[2]] == 1):
            return 9
#           if 2 in back:
#             stunt_ids.append('5*')
#           else:
#             stunt_ids.append('5')
  
      # (1,2,3,4,5) -> (1,2,4,3,5), or (0,1,2,3,4) -> (0,1,3,2,4)
      # right-mid stunt: 6
      if (len1 > 4) & (list2[index[-2]] == len1-3) & (list2[index[-3]] == len1-2):
            return 10
#           if (len1-1) in back:
# #             stunt_ids.append('6*')
# #           else:
# #             stunt_ids.append('6')
  
      # If there's only a 4-man rush, left-mid & right-mid are the same
      # So we have a seventh stunt:
      # (1,2,3,4) -> (1,3,2,4), or (0,1,2,3) -> (0,2,1,4)
      # mid stunt: 7
      if (len1 == 4) & (list2[index[1]] == 2) & (list2[index[2]] == 1):
          if ("2" in back) or (2 in back):
            return 5  # in this case, * means the left defender in back
          else:
            return 6
  
      # (1,2,3,4,5) -> (1,4,3,2,5), or (0,1,2,3,4) -> (0,3,2,1,4)
      # (1,2,3,4,5,6) -> (1,5,3,4,2,6), or (0,1,2,3,4,5) -> (0,4,2,3,1,5)
      # left-bigmid: 8
      if (len1 >= 5) & (list2[index[1]] in range(3, len1-1)):
            return 11
#           if 2 in back:
#             stunt_ids.append('8*')  # left defender in back
#           else:
#             stunt_ids.append('8')
      # (1,2,3,4,5) -> (1,4,3,2,5), or (0,1,2,3,4) -> (0,3,2,1,4)
      # (1,2,3,4,5,6) -> (1,2,5,4,3,6), or (0,1,2,3,4,5) -> (0,1,4,3,2,5)
      # right-bigmid: 9
      if (len1 >= 5) & (list2[index[len1-2]] in range(1, len1-3)):
            return 12
        # if there is an 8 and a 9, then we change it to 10, the bigmid stunt because left-bigmid and right-bigmid could be identical
#         if stunt_ids:
#           if (stunt_ids[-1] in range(8,10)):
#             stunt_ids.pop()
#             stunt_ids.append('10')
#           else:
#             if str(len1-1) in back:
#               stunt_ids.append('9*')  # right defender in back
#             else:
#               stunt_ids.append('9')
#         else:
#             if str(len1-1) in back:
#               stunt_ids.append('9*')  # left defender in back
#             else:
#               stunt_ids.append('9')
      # really just an assortment of middle 
      # can be of length 5 (from above) or 6
      # bigmid stunt: 10

#       # no * for 10
#       if (len1 == 6) & ((list2[index[2]]) == 3):
#         if stunt_ids:
#           if (stunt_ids[-1] in ['8', '8*', '9', '9*']):
#             stunt_ids.pop()
#         stunt_ids.append('10')

#       # (1,2,3,4,5,6,7) -> (1,2,5,4,3,6,7), or (0,1,2,3,4,5,6) -> (0,1,4,3,2,5,6)
#       if (len1 == 7) & (((list2[index[2]]) in range(3,5)) | list2[index[3]] == 4):
#         if stunt_ids:
#           if (stunt_ids[-1] in ['8', '8*', '9', '9*']):
#             stunt_ids.pop()
#         stunt_ids.append('10')
  
      # If none of these stunts match but there is a change, we have an umbrella 'other' category
      # other stunt: 11
      if (not stunt_ids) & (list1 != list2):
          return 13
            
    
    # returns a list of all stunts occurring on the play - stunts are saved as strings
    return stunt_ids

In [79]:
z['stunt'] = None
z['lag_relative_location_list'] = z['relative_location_list'].shift(1)
z['lag_playId'] = z['playId'].shift(1)
for index, row in z.iterrows():
    if ((index != 0) and (z.at[index, 'lag_playId'] == z.at[index, 'playId'])) & (z.at[index, 'lag_relative_location_list'] != z.at[index, 'relative_location_list']):
        z.at[index, 'stunt'] = 1
    else:
        z.at[index, 'stunt'] = 0

In [88]:
z['stunt_class'] = z[0:].apply(lambda row: identify_stunt(row), axis=1)

In [178]:
test_pos = group_and_create_column(z, 'time', 'npos')
test = pd.merge(z, test_pos, on = "time", how = "outer")

In [179]:
test['blitz'] = None

for index, row in test.iterrows():
    plist = test.at[index, 'npos_list']
    
    if ((len(plist) > 4) or (3 in plist) or (2 in plist)):
        test.at[index, 'blitz'] = 1
    else:
        test.at[index, 'blitz'] = 0

In [180]:
test[test['blitz'] == 0]

,gameId,playId,nflId,time,playDirection,x,relative_location,relative_location_list,npos,all_front,all_back,stunt,lag_relative_location_list,lag_playId,stunt_class,npos_list,blitz
288,2021090900,187,35441.0,2021-09-10T00:29:16.700,left,78.41,4.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 1, 1]",0
289,2021090900,187,35454.0,2021-09-10T00:29:16.700,left,78.69,1.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 1, 1]",0
290,2021090900,187,38588.0,2021-09-10T00:29:16.700,left,75.29,2.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 1, 1]",0
291,2021090900,187,46081.0,2021-09-10T00:29:16.700,left,77.89,3.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 1, 1]",0
292,2021090900,187,35441.0,2021-09-10T00:29:16.800,left,78.70,4.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 1, 1]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870193,2021110100,4433,52585.0,2021-11-02T03:20:24.000,right,22.15,2.0,"[3.0, 1.0, 4.0, 2.0]",1,0,0,0,"[3.0, 1.0, 4.0, 2.0]",4433.0,0,"[1, 1, 1, 1]",0
870194,2021110100,4433,42406.0,2021-11-02T03:20:24.100,right,24.12,3.0,"[3.0, 1.0, 4.0, 2.0]",1,0,0,0,"[3.0, 1.0, 4.0, 2.0]",4433.0,0,"[1, 1, 1, 1]",0
870195,2021110100,4433,43326.0,2021-11-02T03:20:24.100,right,22.71,1.0,"[3.0, 1.0, 4.0, 2.0]",1,0,0,0,"[3.0, 1.0, 4.0, 2.0]",4433.0,0,"[1, 1, 1, 1]",0
870196,2021110100,4433,43338.0,2021-11-02T03:20:24.100,right,23.63,4.0,"[3.0, 1.0, 4.0, 2.0]",1,0,0,0,"[3.0, 1.0, 4.0, 2.0]",4433.0,0,"[1, 1, 1, 1]",0


In [181]:
test.iloc[270:300]

,gameId,playId,nflId,time,playDirection,x,relative_location,relative_location_list,npos,all_front,all_back,stunt,lag_relative_location_list,lag_playId,stunt_class,npos_list,blitz
270,2021090900,187,38588.0,2021-09-10T00:29:16.200,left,73.42,2.0,"[4.0, 1.0, 2.0, 3.0]",2,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
271,2021090900,187,46081.0,2021-09-10T00:29:16.200,left,76.36,3.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
272,2021090900,187,35441.0,2021-09-10T00:29:16.300,left,77.00,4.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
273,2021090900,187,35454.0,2021-09-10T00:29:16.300,left,77.86,1.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
274,2021090900,187,38588.0,2021-09-10T00:29:16.300,left,73.69,2.0,"[4.0, 1.0, 2.0, 3.0]",2,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
275,2021090900,187,46081.0,2021-09-10T00:29:16.300,left,76.66,3.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
276,2021090900,187,35441.0,2021-09-10T00:29:16.400,left,77.36,4.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
277,2021090900,187,35454.0,2021-09-10T00:29:16.400,left,78.10,1.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
278,2021090900,187,38588.0,2021-09-10T00:29:16.400,left,74.02,2.0,"[4.0, 1.0, 2.0, 3.0]",2,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1
279,2021090900,187,46081.0,2021-09-10T00:29:16.400,left,76.98,3.0,"[4.0, 1.0, 2.0, 3.0]",1,0,0,0,"[4.0, 1.0, 2.0, 3.0]",187.0,0,"[1, 1, 2, 1]",1


In [182]:
def identify_blitz(row):
    if row['blitz'] == 0:
        return 0
    
    pos_list = row["npos_list"]
    front_rusher = 0
    off_ball = 0
    for i in pos_list:
        if i == 1:
            front_rusher += 1
        if i == 2:
            off_ball += 1
    
    #corner blitz
    if 3 in pos_list:
        return 1
    
    
    
    #five front rushers
    if ((front_rusher == 5) and (off_ball == 0)):
        return 2
    
    #six front rushers
    if ((front_rusher == 6) and (off_ball == 0)):
        return 3
    
    #seven front rushers
    if ((front_rusher == 7) and (off_ball == 0)):
        return 4
    
    
    #4 up and one back
    if ((front_rusher == 4) and (off_ball == 1)):
        return 5
    
    #4 up and two back
    if ((front_rusher == 4) and (off_ball == 2)):
        return 6
    
    #5 up and one back
    if ((front_rusher == 5) and (off_ball == 1)):
        return 7
    
    #5 up and 2 back 
    if ((front_rusher == 5) and (off_ball == 2)):
        return 8
    
    #6 up and 1 back
    if ((front_rusher == 6) and (off_ball == 1)):
        return 9
    
    return 10

In [183]:
test['blitz_class'] = test[0:].apply(lambda row: identify_blitz(row), axis=1)

In [184]:
test.at[0,'stunt_class'] = 0
test['stunt_class'] = test['stunt_class'].replace(['Error: mismatching lists'], 0)

In [185]:
test['blitz'] = test.groupby(['gameId','playId'])['blitz'].transform(lambda x: 1 if x.any() else 0)

In [186]:
test['blitz_class'] = test.groupby(['gameId','playId'])['blitz_class'].transform(lambda x: x.max())

In [187]:
x = test.groupby(['gameId','playId']).agg({'blitz': ['mean']}).reset_index()
x['blitz_class'] = (test.groupby(['gameId','playId']).agg({'blitz_class': ['mean']}).reset_index())['blitz_class']

In [188]:
test['stunt'] = test.groupby(['gameId','playId'])['stunt'].transform(lambda x: 1 if x.any() else 0)

In [193]:
test['stunt_class'] = test.groupby(['gameId','playId'])['stunt_class'].transform(lambda x: min(filter(lambda y: y != 0, x)) if x[x != 0].nunique() > 1 else x[x != 0].unique()[0] if x[x != 0].nunique() == 1 else x.iloc[0])

In [194]:
x['stunt'] = (test.groupby(['gameId','playId']).agg({'stunt': ['mean']}).reset_index())['stunt']
x['stunt_class'] = (test.groupby(['gameId','playId']).agg({'stunt_class': ['mean']}).reset_index())['stunt_class']

In [200]:
plays = pd.read_csv(r'C:\Users\18172\Desktop\Big data bowl\plays.csv')

In [204]:
f = pd.merge(plays, x, on = ['gameId','playId'], how = "right")

C:\Users\18172\AppData\Local\Temp\ipykernel_13112\1515449101.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  f = pd.merge(plays, x, on = ['gameId','playId'], how = "right")
C:\Users\18172\AppData\Local\Temp\ipykernel_13112\1515449101.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  f = pd.merge(plays, x, on = ['gameId','playId'], how = "right")


In [205]:
f.to_csv(r'C:\Users\18172\Desktop\Big data bowl\finalplays.csv')